In [16]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster('LP_1487')
# cld = ClusterDias('Berkeley_97')
# plot = cl.plot_cmd(isochrones=[Isochrone(cld)])
plot = cl.plot_cmd()
show_annotation_manager(plot)


In [4]:
cl.theoretical_isochrone()['Gmag'].min()

19.562255137335207

In [142]:
def latex_text(cluster, n_members=None):
    cld = ClusterDias(cluster.name)
    ra, dec = cluster.skycoord.ra, cluster.skycoord.dec
    ra_str, dec_str = ra.to_string(format='latex')[1:-1], dec.to_string(format='latex')[1:-1]
    dist_str = str(round(cluster.distance.value,1)) + r"\pm" + str(round(cluster.e_distance.value,1))
    
    intro_text = rf"""\subsection{{Introduction}}
    {cluster.name.replace('_', ' ')} is located at $\alpha = {ra_str}, \delta = {dec_str}$ at a distance of ${dist_str}$ pc.
    {cluster.N} out of the {cld.N} mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{{tab:{cluster.name}_kinematics}}. Some of the brightest memebrs fo the cluster are shown in table \ref{{tab:{cluster.name}_members}}.
    
    \subsection{{CMD and proper motion}}
    The parameters of the cluster's isochrone ...
    Using these parameters, the CMD plot for the cluster is shown in figure \ref{{fig:{cluster.name}_cmd_pm}} in the left. The right plot in the figure shows the proper motion diagram showing that the runaway moves with a proper motion that is different from the cluster.
    
    \begin{{figure}}
    \centering
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_cmd.pdf}}
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_pm.pdf}}
    \caption{{\\ 
    \textit{{Left}}: The CMD showing the temperature estimate of the runaway. The isochrone for the temperature estimate is shown in blue \\
    \textit{{Right}}: Proper motion diagram depicting that the star is an outlier in proper motion in the local rest frame.}}
        \label{{fig:{cluster.name}_cmd_pm}}
    \end{{figure}}    

    \subsection{{Traceback}}
    
    The runaway star's motion relative to the cluster is shown in figure \ref{{fig:{cluster.name}_traceback.pdf}}.
    
    \begin{{figure}}
    \centering
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_traceback_clean.pdf}}
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_pm.pdf}}
    \caption{{\\ 
    \textit{{Left}}: The runaway star ...'s motion with respect to the cluster. The four green lines depict the four extreme cases of the proper motion considering the errors and the green ellipse represents the possible positions of the star 100 kyr ago. \\
    \textit{{Right}}: Proper motion diagram depicting that the star is an outlier in proper motion in the local rest frame.}}
        \label{{fig:{cluster.name}_traceback_clean&pm}}
    \end{{figure}}
    
    \subsection{{Traceback}}
    """
    
    # Members table generation
    table = cluster.mymembers
    table.sort('Gmag')
    table = table[:n_members]
    
    formatted_distances = []
    formatted_parallaxes = []
    formatted_gmags = []
    formatted_bprps = []
    
    for row in table:
        distance = row['rgeo']
        plx = row['Plx']
        e_plx = row['e_Plx']
        upper_error = row['B_rgeo'] - row['rgeo']
        lower_error = -row['b_rgeo'] + row['rgeo']
        gmag = row['Gmag']
        e_gmag = row['e_Gmag']
        bprp = row['BP-RP']
        e_bprp = row['e_BP-RP']
        
        formatted_distance = f"${distance:.0f}^{{+{upper_error:.0f}}}_{{-{lower_error:.0f}}}$"
        formatted_parallax = f"${plx:.4f}\pm{e_plx:.4f}$"
        formatted_gmag = f"${gmag:.3f}\pm{e_gmag:.3f}$"
        formatted_bprp = f"${bprp:.3f}\pm{e_bprp:.3f}$"
        
        formatted_distances.append(formatted_distance)
        formatted_parallaxes.append(formatted_parallax)
        formatted_gmags.append(formatted_gmag)
        formatted_bprps.append(formatted_bprp)

    table['formatted_distance'] = formatted_distances
    table['formatted_parallax'] = formatted_parallaxes
    table['formatted_gmag'] = formatted_gmags
    table['formatted_bprp'] = formatted_bprps
    
    t_dict_members = {
        'Gaia DR3 Source': table['Source'],
        r"$r_{\text{geo}}$ (pc)": table['formatted_distance'],
        r"$\pi$ (mas)": table['formatted_parallax'],
        r"$G$ (mag)": table['formatted_gmag'],
        r"$G_{\text{BP}}-G_{\text{RP}}$ (mag)": table['formatted_bprp']
    }
    
    latexdict_members = {
        'tabletype': 'table*',
        'tablealign': 'h',
        'header_start': r'\hline',
        'data_start': r'\hline',
        'data_end': r'\hline', 
        'caption': f'Selected members of {cluster.name.replace("_"," ")}',
        'preamble': r'\label{tab:' + f'{cluster.name}_members' + '}'
    }
    
    astropy_table_members = Table(t_dict_members)
    members_output_filename = 'members_text.txt'
    
    astropy.io.ascii.write(astropy_table_members, format='latex', output=members_output_filename, overwrite=True, latexdict=latexdict_members)
    
    with open(members_output_filename, 'r+') as f:
        lines = f.readlines()
        lines[1], lines[2] = lines[2], lines[1]
        f.seek(0)
        f.writelines(lines)
        members_table_text = ''.join(lines)
    
    os.remove(members_output_filename)
    kinematics_table_text = generate_kinematics_latex(cluster.name)
    # Combine intro_text, kinematics_table_text, and members_table_text
    full_text = intro_text + "\n\n" + kinematics_table_text + "\n\n" + members_table_text
    
    filename = f"{cluster.name}.tex"
    
    with open(filename, 'w') as file:
        file.write(full_text)
    
    return full_text

# Example usage
cl = Cluster('LP_1487')
print(latex_text(cl, n_members=10))


\subsection{Introduction}
    LP 1487 is located at $\alpha = 160^\circ29{}^\prime32.64{}^{\prime\prime}, \delta = -58^\circ22{}^\prime22.44{}^{\prime\prime}$ at a distance of $2710.0\pm177.0$ pc.
    29 out of the 195 mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{tab:LP_1487_kinematics}. Some of the brightest memebrs fo the cluster are shown in table \ref{tab:LP_1487_members}.
    
    \subsection{CMD and proper motion}
    The parameters of the cluster's isochrone ...
    Using these parameters, the CMD plot for the cluster is shown in figure \ref{fig:LP_1487_cmd_pm} in the left. The right plot in the figure shows the proper motion diagram showing that the runaway moves with a proper motion that is different from the cluster.
    
    \begin{figure}
    \centering
    \includegraphics[width=0.49\linewidth]{Results/LP_1487/LP_1487_cmd